In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 29 12:12:18 2020

@author: Naveen
"""
import numpy as np
#from classifiers import seqDenseNN
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

%run classifiers.ipynb

def splitData(xTrain, yTrain, labeledSize, selectionType):
    idx = np.array([])
    if (selectionType == "Group"):
        size = int(labeledSize/10)
        idx = np.append(idx, np.random.choice(np.where(yTrain == 0)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 1)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 2)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 3)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 4)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 5)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 6)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 7)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 8)[0], 
                                              size, replace = False))
        idx = np.append(idx, np.random.choice(np.where(yTrain == 9)[0], 
                                              size, replace = False))
    else:
        idx = np.random.choice(yTrain.shape[0], labeledSize, replace = False)
    idx = idx.astype(int)
    labeledXTrain = xTrain[idx, ]
    labeledYTrain = yTrain[idx, ]
    
    unlabeledXTrain = xTrain[[i for i in range(xTrain.shape[0]) if i not in idx], ]
    unlabeledYTrain = yTrain[[i for i in range(xTrain.shape[0]) if i not in idx], ]
    return([labeledXTrain, labeledYTrain, unlabeledXTrain, unlabeledYTrain])
    
def selfTrainModel(xTrain, yTrain, batchSize, maxIter, pThresh, labeledSize, selectionType):
    # Assumes that the classification model can also calculate class probabilities
    
    # 1. Randomly classify data as labeled and unlabeled according to label size
    data = splitData(xTrain, yTrain, labeledSize, selectionType)
    labeledXTrain = data[0]
    labeledYTrain = data[1]    
    unlabeledXTrain = data[2]
    unlabeledYTrain = data[3]
    
    labeledYTrain = to_categorical(labeledYTrain, 10)
    unlabeledYTrain = to_categorical(unlabeledYTrain, 10)
    
    unlabeledData = np.copy(unlabeledYTrain)
    seqDenseModel = seqDenseNN(labeledXTrain)
    
    
    count = 0
    wrongLabePct = []
    
    # Check if there are any more unlabeled datapoints left or if the number of iterations are done
    while((count <= maxIter) and (len(unlabeledData) != 0)):
        print(count)
        print(len(unlabeledData))
        # 2. Run a CNN on labeled data and get predicted labels and probabilities
        seqDenseModel.fit(labeledXTrain, labeledYTrain, batch_size = batchSize, epochs = 5,  
                     validation_split = 0.1, verbose = 2)
        unlabeledYHat = seqDenseModel.predict_classes(unlabeledXTrain, batch_size = batchSize)
        predProbs = seqDenseModel.predict(unlabeledXTrain, batch_size = batchSize)
        
        # 3. Choose datapoints with high label probabilities and add them to labaled set
        highProbIdx = np.where(predProbs > pThresh)[0]
        labeledXTrain = np.vstack((labeledXTrain, unlabeledXTrain[highProbIdx, :]))
        labeledYTrain = np.vstack((labeledYTrain, unlabeledYTrain[highProbIdx, :]))
        
        # Check how much of the unlabeled data that was added to labeled data was wrongly classified
        unlabeledYHat = unlabeledYHat[highProbIdx]        
        cMat = confusion_matrix(np.argmax(unlabeledYTrain[highProbIdx, :], axis = -1), unlabeledYHat)
        errorPct = 1 - sum(np.diag(cMat))/sum(sum(cMat))
        wrongLabePct = np.append(wrongLabePct, errorPct)
        
        # Remove unlabeled data that was added to labeled data from labeled data
        unlabeledXTrain = unlabeledXTrain[[i for i in range(unlabeledXTrain.shape[0]) \
                                           if i not in highProbIdx], ]
        unlabeledYTrain = unlabeledYTrain[[i for i in range(unlabeledYTrain.shape[0]) \
                                           if i not in highProbIdx], ]
        unlabeledData = np.copy(unlabeledYTrain)
        count += 1
    return([seqDenseModel, wrongLabePct])

def fullModel(xTrain, yTrain, batchSize):
    seqDenseModel = seqDenseNN(xTrain)
    yTrain = to_categorical(yTrain, 10)
    seqDenseModel.fit(xTrain, yTrain, batch_size = batchSize, epochs = 5, 
                 validation_split = 0.1, verbose = 2)
    return(seqDenseModel)


Using TensorFlow backend.
